# Projeto 1 - Ciência dos Dados

Nome: Vitor Fortes Giuliano Riccetti 

Nome: Rodrigo Paoliello de Medeiros

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'Chuteira_Puma.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Chuteira_Puma.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
# Faz a leitura do excel 
train = pd.read_excel(filename)
train.head(5)

train["Relevante"]= train["Relevante"].astype(str)
train["Irrelevante"]= train["Irrelevante"].astype(str)

train.head(5)

,Treinamento,Relevante,Irrelevante
0,NaN,"@puma_br @palmeiras meu time, nossa cria, noss...",@neymarjr @puma chamou na chuteira clássica
1,NaN,doido que as pelada na arena volta pra eu lanç...,"porra ney, era pra inaugurar a chuteira da pum..."
2,NaN,essa chuteira da puma😍😍😍 https://t.co/ra7ojthqxs,@neymarjr @puma chuteira preta! ja posso iludi...
3,NaN,@neymarjr @puma tem uma chuteira mais bonitinh...,"bicho, o neymar recusou renovação de contrato ..."
4,NaN,@gaboquesabe nunca vi uma chuteira top tier 1 ...,@neymarjr @puma chuteira preta pai agora a cop...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevante,Irrelevante
0,NaN,@gathlos de níver atrasado quero uma chuteira ...,"""o rei está de volta!""\n\na puma confirmou hoj..."
1,NaN,@neymarjrdepre @neymarjr @puma essa chuteira é...,"@bergonsi_carlos chuteira preta da puma, é ami..."
2,NaN,era só essa chuteira da puma na minha vida,"@bernardoqac chuteira preta da puma e barões, ..."
3,NaN,@manassescruz1 @juancrf19 @futebol_info @puma_...,@danilindo____ @neymarjrdepre @breninhoo___ @n...
4,NaN,só a chuteira da puma na minha vida mesmo.,@futtmais posta as fotos d ney com a nova chut...


In [5]:
# dividimos em duas categorias relevante e irrelevante:

Relevantes = ' '.join(train.Relevante)

Irrelevantes = ' '.join(train.Irrelevante)

In [29]:
# Função para limpar a string
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    text = re.sub(r'_', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\n', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    punctuation = '[!-.:?;@,“”_]' # Note que os sinais [] são delimitadores de um conjunto.

    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [30]:
# Retira as pontuações e os links
cleanup(Relevantes)

#cleanup(Irrelevantes)

'puma br palmeiras meu time nossa cria nossa puma br poderia me presentear com uma chuteira pra fechar o squad rsrsrs boyah doido que as pelada na arena volta pra eu lançar uma chuteira da puma igual menino ney 😅 essa chuteira da puma😍😍😍  neymarjr puma tem uma chuteira mais bonitinha ñ ney gaboquesabe nunca vi uma chuteira top tier 1 da puma neymar assinando com a puma e lançando uma chuteira horrorosa essa chuteira preta básica da puma é clássica dmsss  guiwrid neymarjrdepre eu falo em chuteira mesmo já testei adidas puma mas a nike é a única que encaixa perfeitamente palmeiras puma br chuteira feia yurifirme neymarjr pumafootball puma acabando com meu sonho de ter uma chuteira puma agora vai custar 2 mil reais kkkkkk minha opinião chuteira da puma é muito fatela em termos de tudo nem com o dinheiro q o ney ganha da puma eu usaria essa chuteira feia vsf  seloco tô apaixonada nessa chuteira preta da puma do neymar 🤤🤤 chuteira preta kkkk ue ate eu vou usar puma agr 😂  liventy222 pkakaro

In [16]:
R = pd.Series(Relevantes.split())

In [23]:
R.value_counts(ascending=False)

chuteira                   140
puma                       105
da                          99
a                           57
que                         42
                          ... 
porfavor                     1
simples                      1
amarelo                      1
silhueta                     1
https://t.co/wynmzpgc7m      1
Length: 695, dtype: int64

In [28]:
Relevantes.count()

16

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**